# Metadata analyse notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
df = pd.read_excel('ledenenquete_over_heel_de_mens.xlsx', skiprows=2, names = ['Serienummer', 'SID', 'Submitted Time', 'Verwerkingstijd',
       'Modified Time', 'Kladversie', 'IP-adres', 'UID', 'Gebruikersnaam',
       'Naam', 'Email', 'Afdeling',
       'indruk', 'ontwikkelingen', 'redenen', 'meegeven']) 

In [ ]:
df.columns

In [ ]:
df.info()

- 5882 keer is de enquete gestart,
- 2796 is er ingediend
- 3225 keer 'indruk' vraag ingevuld
- 2781 keer 'ontwikkel' vraag ingevuld
- 2890 keer 'reden vraag ingevuld
- 2814 keer 'meegeven aan commissie' vraag ingevuld

In [ ]:
len(set(df['Naam']))

In [ ]:
df_copy = df.copy()
df_copy.Afdeling = df_copy.Afdeling.fillna('Onbekend')
df_copy.Naam = df_copy.Naam.fillna('Socialist')
df_incomplete = df_copy[df_copy.Kladversie==1]
print(f'ingevulde enquetes: {len(df_copy)}')
print(f'waarvan kladversies: {len(df_incomplete)}')

    
agg = df_incomplete.pivot_table(columns=['Kladversie', 'Naam'], aggfunc='size', fill_value=0)
tot = 0
for i in sorted(list(agg.unique())):
    size = df_incomplete.pivot_table(columns=['Kladversie', 'Naam'], aggfunc='size', fill_value=0)
    size = size[size == i]
    tot = tot + len(size)
    print(f'{len(size)} aantal leden gebruikte {i} kladversies; subtotaal = {tot}')

    

# Unieke bijdrages?

In [ ]:
# set onbekende afdelingen naar bekende afdelingen als de naam meerdere keren voorkomt
df_copy = df.copy()
df_copy.Afdeling = df_copy.Afdeling.fillna('@' + df_copy['IP-adres'])
#names = list(set(df_copy.Naam))
mail = list(set(df_copy.Email))
for email in mail:
    indices = df_copy[df_copy.Email == email].index.tolist()
    try:
        city = [x for x in list(set(df_copy[df_copy.Email == email].Afdeling)) if not x.startswith('@')][0]
    except:
        city = None
    df_copy.loc[indices,'Afdeling'] = city
    
df_copy.Afdeling = df_copy.Afdeling.fillna('@' + df_copy['IP-adres'])


In [ ]:
df_unique = df_copy.drop_duplicates(subset = ['Naam', 'Afdeling'], keep = 'last')
df_unique = df_unique.drop_duplicates(subset = ["Email"], keep='first')
print(f'aantal unieke personen: {len(df_unique)}')

In [ ]:
df_unique = df_unique[df_unique.Kladversie==0]
df_unique.info()

In [ ]:
127/2662

In [ ]:
df_complete = df[df.Verwerkingstijd.notnull()]
df_complete.Afdeling = df_complete.Afdeling.fillna('Onbekend')
df_complete.Naam = df_complete.Naam.fillna('Socialist')
print(df_complete.info())

print(f'compleet ingevulde enquetes: {len(df_complete)}')

dup = df_complete[df_complete.duplicated(subset = ['Naam', 'Afdeling'], keep=False)]
dup.to_excel("duplicates.xlsx") 


In [ ]:
df.isnull().sum()

- 3086 leden hebben de enquete niet afgemaakt
- 5880 leden hebben geen gebruikersnaam opgegeven
- 72 leden hebben geen naam 
- 74 leden hebben geen email 
- 990 hebben geen afdeling

In [ ]:
#Plot van alle leden die iets ingevuld hebben

In [ ]:
plt.close('all')
fig = plt.figure()
freq = df['Afdeling'].value_counts()
freq.plot(kind="bar", title = "participerende leden per afdeling", xlabel = 'Afdeling', color = 'red', alpha=0.5, 
          figsize=(20,5))
plt.tight_layout()
plt.savefig('all.jpg')


In [ ]:
freq = freq.reset_index()
print(np.array(freq.sort_values(['index'])))

In [ ]:
df_complete = df[df.Verwerkingstijd.notnull()]
print(df.shape)
print(df_complete.shape)

In [ ]:
plt.close('all')
fig = plt.figure()
freq = df_complete['Afdeling'].value_counts()
print(f'aantal afdelingen {len(freq)}')
freq.plot(kind="bar", title = "participerende leden per afdeling compleet ingevuld", xlabel = 'Afdeling', color = 'red', alpha=0.9, 
          figsize=(20,5))
plt.tight_layout()

In [ ]:
freq = freq.reset_index()
print(np.array(freq.sort_values(['index'])))